In [1]:
import query_ris

from ipyleaflet import Map, basemaps, basemap_to_tiles, DrawControl, GeoJSON, Marker, MarkerCluster, AwesomeIcon
from ipydatetime import DatetimePicker
from IPython.core.display import display
import ipyleaflet
from ipywidgets import widgets, Layout
from datetime import datetime, timezone
import dateutil
import dateutil.parser
import pytz

import plotly.io as pio

%load_ext ipycache

import warnings
warnings.filterwarnings('ignore')

/home/dboulang/anaconda3/lib/python3.8/site-packages/IPython/config.py:12: ShimWarning: The `IPython.config` package has been deprecated since IPython 4.0. You should import from traitlets.config instead.
  warn("The `IPython.config` package has been deprecated since IPython 4.0. "
/home/dboulang/anaconda3/lib/python3.8/site-packages/ipycache.py:17: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils.traitlets import Unicode


In [5]:
def get_start_date():
    return datetime(2000, 1, 1, 0, 0, tzinfo=pytz.utc)

def get_end_date():
    return datetime.now(timezone.utc)

# Launch search
def on_search_clicked(b):
    with out:
        out.clear_output()
        start = datetime_picker_start.value.strftime("%Y-%m-%d")
        end = datetime_picker_end.value.strftime("%Y-%m-%d")
        print("QUERY\nbbox=" + str(bbox) + "\nbetween " + str(start) + " and " + str(end)+ "\nVariables: " + str(list_variables.value))
        print(query_datasets())
    with datasets_panel:
        print("vivible")
        list_datasets = selection_datasets_list(query_datasets(datetime_picker_start.value.strftime("%Y-%m-%dT%H:%M:%S"), datetime_picker_end.value.strftime("%Y-%m-%dT%H:%M:%S"), list_variables.value, bbox))
        datasets_panel = widgets.HBox([list_datasets])
        datasets_panel.layout.visibility = 'visible'

# Clear output panel
def mk_clear_button(target, action_name):
    button = widgets.Button(description=action_name.replace('_', ' '))
    action = getattr(target, action_name)
    button.on_click(lambda *a: action())
    return button

def mk_search_button():
    button = widgets.Button(description="Search")
    button.on_click(on_search_clicked)
    return button

# Display coordinates
def handle_interaction(**kwargs):
    if kwargs.get('type') == 'mousemove':
        coords = kwargs.get('coordinates')
        lat_label.value = str(coords[0]) 
        lon_label.value = str(coords[1])

def handle_click(**kwargs):
    with out:
        print(kwargs)
        print(type(kwargs))
        # kwargs['feature']['properties'][foobar] = 10 
        
def on_draw_handler(draw, action, geo_json):
    with out:
        out.clear_output()
        for i in map_panel.layers:
            if type(i) == ipyleaflet.GeoJSON:
                map_panel.remove_layer(i)
        bounds = geo_json['geometry']['coordinates'][0]
        ll = bounds[0]
        ur = bounds[2]
        corners = [ll, ur]
        global bbox
        bbox = [item for sublist in corners for item in sublist]
        searchbox = GeoJSON(data=geo_json)
        searchbox.on_click(handle_click)
        map_panel.add_layer(searchbox)
        draw.clear_rectangles()
        
def selection_variables_list(values):
    return widgets.SelectMultiple(options=values,
    rows=10,
    description='Variables',
    disabled=False,
    layout=Layout(width='800px',)
)    

def selection_datasets_list(values):
    return widgets.SelectMultiple(options=values,
    rows=30,
    description='Select datasets to plot',
    disabled=False,
    layout=Layout(width='1600px',)
)      

ri_icons = { 'actris': AwesomeIcon(
    name='map-marker',
    marker_color='blue',
    icon_color='black',
    spin=False
), 'iagos': AwesomeIcon(
    name='plane',
    marker_color='green',
    icon_color='black',
    spin=False
), 'icos': AwesomeIcon(
    name='map-marker',
    marker_color='red',
    icon_color='black',
    spin=False
), 'sios': AwesomeIcon(
    name='map-marker',
    marker_color='orange',
    icon_color='black',
    spin=False
) }

def display_platforms(platforms, m):
    markers=[]
    for ri in ri_icons.keys():
        for platform in platforms[ri]:
            center = (platform['latitude'], platform['longitude'])
            marker = Marker(icon=ri_icons[ri], location=center, draggable=False, title=(platform['short_name'] + " - " + ri))
            markers.append(marker)
    marker_cluster = MarkerCluster(markers=markers)
    m.add_layer(marker_cluster);

In [7]:
variables = query_ris.load_variables()
print(variables)

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [7]:
%%cache mycache.pkl platforms variables
platforms = query_ris.load_platforms()
variables = query_ris.load_variables()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

ValueError: Variable(s) 'platforms', 'variables' could not be found in the interactive namespace

In [ ]:
# Temporal extent
datetime_picker_start = DatetimePicker(value=get_start_date())
datetime_picker_end = DatetimePicker(value=get_end_date())

# Output panel
out = widgets.Output(layout=widgets.Layout(width='50%', 
                               height='400px', 
                               overflow_y='scroll',
                               border='1px solid black'))
clear_output = mk_clear_button(out, 'clear_output')
search = mk_search_button()

# Map panel
map_panel = ipyleaflet.Map(layers=(basemap_to_tiles(basemaps.Stamen.Terrain), ), center=(49, 11), zoom=4)
controls = ipyleaflet.LayersControl(position='topright')
lat_label = widgets.Label()
lon_label = widgets.Label()
map_panel.add_control(controls)
map_panel.on_interaction(handle_interaction)
global bbox
bbox = [-180, -90, 180, 90]

list_variables = selection_variables_list(get_ECV(variables))
list_datasets = selection_datasets_list([])

#Draw panel for selection on map
draw = ipyleaflet.DrawControl(
    edit=True,
    remove=True,
    circlemarker={},
    marker={},
    circle={},    
    polyline={},
    polygon = {},
    rectangle={'shapeOptions': {}})
draw.on_draw(on_draw_handler)
map_panel.add_control(draw)

display_platforms(platforms, map_panel)

datasets_panel = widgets.HBox([list_datasets])
datasets_panel.layout.visibility = 'hidden'

dashboard = widgets.VBox([ 
                    widgets.HBox([datetime_picker_start, datetime_picker_end]), 
                    widgets.HBox([list_variables]), 
                    widgets.HBox([clear_output, search]),
                    widgets.HBox([map_panel, out]), widgets.HBox([lon_label, lat_label]),
                    datasets_panel
])

dashboard